# 数据预处理

In [171]:
import pandas as pd
import matplotlib.pyplot as plt

In [172]:
filepath = "超市.xls"

## 数据总数与特征

In [173]:
df = pd.read_excel(filepath)

print("数据总数：" + str(df.shape[0]))
print("特征总数：" + str(len(df.columns.tolist())))

数据总数：9935
特征总数：20


In [174]:
# 缺失值检查
missing_values = df.isnull()

if missing_values.any().any():
    print("DataFrame中存在缺失值")
else:
    print("DataFrame中没有缺失值")

DataFrame中没有缺失值


## 分别对各特征进行预处理

In [175]:
print(df.describe())

               利润率     记录数            利润                           发货日期  \
count  9935.000000  9935.0   9935.000000                           9935   
mean      0.085460     1.0    213.632612  2015-05-09 08:21:29.984901632   
min      -4.000000     1.0  -7978.000000            2013-01-03 00:00:00   
25%       0.020000     1.0      7.000000            2014-06-18 00:00:00   
50%       0.170000     1.0     74.000000            2015-07-16 00:00:00   
75%       0.340000     1.0    276.000000            2016-05-16 00:00:00   
max       0.500000     1.0  10108.000000            2017-01-06 00:00:00   
std       0.465521     0.0    856.568912                            NaN   

                折扣           数量                           订单日期  
count  9935.000000  9935.000000                           9935  
mean      0.107111     3.768093  2015-05-05 09:25:25.153497600  
min       0.000000     1.000000            2013-01-01 00:00:00  
25%       0.000000     2.000000            2014-06-13 00:00:00  

### 订单ID

In [176]:
print(df['订单 Id'])

0       US-2015-1357144
1       CN-2015-1973789
2       CN-2015-1973789
3       US-2015-3017568
4       CN-2014-2975416
             ...       
9930    CN-2015-4318875
9931    CN-2015-4318875
9932    CN-2015-4318875
9933    CN-2015-4318875
9934    CN-2012-3557528
Name: 订单 Id, Length: 9935, dtype: object


In [177]:
# 拆分成三个子特征
df[['订单国家','订单年份', '订单编号']] = df['订单 Id'].str.split('-', expand=True)
# 删除原特征
df.drop(columns = ['订单 Id', '订单年份'], inplace=True)

### 产品名称

In [178]:
df[['产品名称', '产品描述']] = df['产品名称'].str.split(', ', expand=True)

# 显示结果
print(df[['产品名称', '产品描述']].head())

             产品名称 产品描述
0      Fiskars 剪刀   蓝色
1  GlobeWeis 搭扣信封   红色
2  Cardinal 孔加固材料   回收
3    Kleencut 开信刀   工业
4  KitchenAid 搅拌机   黑色


In [179]:
print(df['产品描述'].drop_duplicates().shape)

(30,)


In [180]:
# 定义一个函数，用于提取真正的名称
def extract_product_name(row):
    return row['产品名称'].lower().replace(row['制造商'].lower() + ' ', '').strip()

# 使用 apply 方法将函数应用到每一行
df['产品名称'] = df.apply(extract_product_name, axis=1)

# 显示结果
print(df[['产品名称']].head())

    产品名称
0     剪刀
1   搭扣信封
2  孔加固材料
3    开信刀
4    搅拌机


In [181]:
# 检查
rows_with_spaces = df[df['产品名称'].str.contains(' ')]

# 显示结果
print(rows_with_spaces[['制造商', '产品名称']].drop_duplicates())

             制造商              产品名称
304   Rubbermaid     rubvermaid 灯泡
490   Rubbermaid     rubvermaid 框架
504   Rubbermaid     rubvermaid 门挡
951   Rubbermaid  rubvermaid 分层置放架
1066  Rubbermaid     rubvermaid 闹钟
1385  Rubbermaid     rubvermaid 相框


In [182]:
# 处理rubvermaid
def extract_product_name(row):
    return row['产品名称'].lower().replace('rubvermaid' + ' ', '').strip()

# 使用 apply 方法将函数应用到每一行
df['产品名称'] = df.apply(extract_product_name, axis=1)

# 显示结果
print(df[['产品名称']].head())

    产品名称
0     剪刀
1   搭扣信封
2  孔加固材料
3    开信刀
4    搅拌机


In [183]:
# 检查
rows_with_spaces = df[df['产品名称'].str.contains(' ')]

# 显示结果
print(rows_with_spaces[['制造商', '产品名称']].drop_duplicates())

Empty DataFrame
Columns: [制造商, 产品名称]
Index: []


In [184]:
print(df[['产品名称']].drop_duplicates().shape)

(102, 1)


### 利润率&利润

In [185]:
# 利润率
print(df['利润率'])

# 利润
print(df['利润'])

0      -0.47
1       0.34
2       0.13
3      -0.08
4       0.40
        ... 
9930   -0.33
9931    0.08
9932   -0.37
9933   -0.33
9934   -0.48
Name: 利润率, Length: 9935, dtype: float64
0        -61
1         43
2          4
3        -27
4        550
        ... 
9930    -315
9931      36
9932     -88
9933   -1617
9934   -3502
Name: 利润, Length: 9935, dtype: int64


### 记录数

In [186]:
print((df['记录数'] == 1).sum() == df.shape[0])

True


In [187]:
# 值全为1，删除本列
df.drop(columns = ['记录数'], inplace=True)

### 国家

In [188]:
print((df['国家'] == '中国').sum() == df.shape[0])

True


In [189]:
# 值全为中国，删除本列
df.drop(columns = ['国家'], inplace=True)

### 客户名称

In [190]:
# 删除无关列
df.drop(columns = ['客户名称'], inplace=True)

## 数据导出

In [191]:
df.to_excel("preprocessed_supermarket.xlsx", index=False)